In [1]:
import datetime as dt
import pandas as pd
import numpy as np
from scipy import stats
from pandas_datareader import data as pdr

**Step 1: specify date range for analysis**

In [2]:
start = dt.datetime(2021,1,1)
end = dt.datetime.now()
start, end

(datetime.datetime(2021, 1, 1, 0, 0),
 datetime.datetime(2022, 7, 5, 23, 5, 45, 624363))

**Step2: select the stock/tickers you would like to analyse**

In [22]:
stockList = ['CBA', 'NAB', 'WBC', 'ANZ']
stocks = ['^AXJO'] + [i + '.AX' for i in stockList]
stocks

['^AXJO', 'CBA.AX', 'NAB.AX', 'WBC.AX', 'ANZ.AX']

**Step 3: call the Pandas_Datareader  datareader module**

In [23]:
df = pdr.get_data_yahoo(stocks, start, end)
log_returns = np.log(df.Close / df.Close.shift(1)).dropna()
log_returns.head()

Symbols,^AXJO,CBA.AX,NAB.AX,WBC.AX,ANZ.AX
Date,,,,,
2021-01-04,-0.000344,-0.006228,-0.010522,-0.005619,-0.003043
2021-01-05,-0.011258,-0.003973,-0.012417,-0.007714,-0.008306
2021-01-06,0.015752,0.020063,0.027725,0.031507,0.037057
2021-01-07,0.006815,0.012338,0.013365,0.014403,0.008425
2021-01-10,-0.009023,-0.005504,-0.006014,0.000493,0.000419


**Step 4a: directly calculate data**

In [26]:
def calc_beta(df):
    np_array = df.values
    ## market  index is the first column 0
    m = np_array[:,0]
    beta = []
    for ind,col in enumerate(df):
        if ind > 0:
            ## stock returns indexed by ind
            s = np_array[:,ind]
            ## calculate the cov matrix between the stock and market
            covariance = np.cov(s,m)
            beta.append(covariance[0,1] / covariance[1,1])
            
    return pd.Series(beta, df.columns[1:], name = 'Beta')




In [27]:
calc_beta(log_returns)

Symbols
CBA.AX    1.021162
NAB.AX    0.914802
WBC.AX    0.863219
ANZ.AX    0.908976
Name: Beta, dtype: float64

**Step 4b: use linear regression to get coefficient of market and stocks returns**

In [29]:
def regression_beta(df):
    np_array = df.values
    ## market  index is the first column 0
    m = np_array[:,0]
    beta = []
    for ind,col in enumerate(df):
        if ind > 0:
            ## stock returns indexed by ind
            s = np_array[:,ind]
            beta.append(stats.linregress(m,s)[0])
                   
    return pd.Series(beta, df.columns[1:], name = 'Beta')

In [30]:
regression_beta(log_returns)

Symbols
CBA.AX    1.021162
NAB.AX    0.914802
WBC.AX    0.863219
ANZ.AX    0.908976
Name: Beta, dtype: float64

**Step 4c: use matrix algebra to complete linear regression in one line**

In [31]:
def matrix_beta(df):
    # market index
    X = df.values[:,[0]]
    # add an additional columns for itercepept (initialize these values as 1's)
    X = np.concatenate([np.ones_like(X),X], axis =1)
    # apply the matrix algebra for linear regression - closed form solution
    beta = np.linalg.pinv(X.T @ X) @ X.T @ df.values[:,1:] ## '@' stands for 'np.dot()'

    return pd.Series(beta[1], df.columns[1:], name = 'Beta')

In [38]:
beta = matrix_beta(log_returns)

**Step 5: define your ptf and make DataFrame**

In [40]:
units = np.array([100,250,300,400])
ASXprices = df.Close[-1:].values.tolist()[0]
price = np.array([round(price,2) for price in ASXprices[1:]])
value = units*price
weight = [round(val/sum(value),2) for val in value]
beta = round(beta,2)

In [50]:
ptf = pd.DataFrame({
    'Stock': stockList,
    'Direction': 'Long',
    'Type': 'S',
    'Stock Price': price,
    'Price' : price,
    'Units': units,
    'Value': value,
    'Weight': weight,
    'Beta': beta,
    'Weighted Beta': weight * beta
})

ptf

,Stock,Direction,Type,Stock Price,Price,Units,Value,Weight,Beta,Weighted Beta
Symbols,,,,,,,,,,
CBA.AX,CBA,Long,S,91.23,91.23,100,9123.0,0.30,1.02,0.3060
NAB.AX,NAB,Long,S,27.65,27.65,250,6912.5,0.22,0.91,0.2002
WBC.AX,WBC,Long,S,19.67,19.67,300,5901.0,0.19,0.86,0.1634
ANZ.AX,ANZ,Long,S,22.20,22.20,400,8880.0,0.29,0.91,0.2639


**Step 6: what if we have options, let's consider things in terms of Delta**

In [51]:
ptf = ptf.drop(['Weighted Beta','Weight'], axis=1)
ptf['Delta'] = ptf['Units']
ptf

,Stock,Direction,Type,Stock Price,Price,Units,Value,Beta,Delta
Symbols,,,,,,,,,
CBA.AX,CBA,Long,S,91.23,91.23,100,9123.0,1.02,100
NAB.AX,NAB,Long,S,27.65,27.65,250,6912.5,0.91,250
WBC.AX,WBC,Long,S,19.67,19.67,300,5901.0,0.86,300
ANZ.AX,ANZ,Long,S,22.20,22.20,400,8880.0,0.91,400


**Add options to ptf**

In [64]:
Options = [{'option':'CBA0Z8', 'underlying':'CBA', 'price':3.950, 'units': 2, 'delta': 0.627, 'direction': 'Short', 'type': 'Call'},
           {'option':'NABQB9', 'underlying':'NAB', 'price':1.325, 'units': 2, 'delta': -0.425 ,'direction': 'Long', 'type': 'Put'}]

In [65]:
for index, row in enumerate(Options):
    ptf.loc[row['option']] = [row['underlying'], row['direction'], row['type'], ptf.loc[row['underlying']+'.AX', 'Price'], 
                                    row['price'], row['units'], row['price']*row['units']*100, beta[row['underlying']+'.AX'], 
                                    (row['delta']*row['units']* 100 if row['direction'] == 'Long' else -row['delta']*row['units']*100)]

ptf


,Stock,Direction,Type,Stock Price,Price,Units,Value,Beta,Delta
Symbols,,,,,,,,,
CBA.AX,CBA,Long,S,91.23,91.230,100,9123.0,1.02,100.0
NAB.AX,NAB,Long,S,27.65,27.650,250,6912.5,0.91,250.0
WBC.AX,WBC,Long,S,19.67,19.670,300,5901.0,0.86,300.0
ANZ.AX,ANZ,Long,S,22.20,22.200,400,8880.0,0.91,400.0
CBA0Z8,CBA,Short,Call,91.23,3.950,2,790.0,1.02,-125.4
WPLQB9,NAB,Long,Put,27.65,1.325,2,265.0,0.91,-85.0
NABQB9,NAB,Long,Put,27.65,1.325,2,265.0,0.91,-85.0
